# Notebook 3: Mountain Wave Test Case

This notebook will go through an example of a hydrostatic flow over a mountain. This test case is outlined in the paper: Melvin, T., Dubal, M., Wood, N., Staniforth, A., & Zerroukat, M. (2010). An inherently mass‐conserving iterative semi‐implicit semi‐Lagrangian discretization of the non‐hydrostatic vertical‐slice equations. Quarterly Journal of the Royal Meteorological Society: A journal of the atmospheric sciences, applied meteorology and physical oceanography, 136(648), 799-814.

A mountain wave. This follows the example script 'mountain_nonhydrostatic.py'. This will differ from the sk_nonlinear case we saw in Notebook 2, as hydrostatic balance is no longer enforced.

We begin by importing the required libraries and functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, VectorFunctionSpace,
                       PeriodicIntervalMesh, ExtrudedMesh, SpatialCoordinate,
                       exp, pi, cos, Function, conditional, Mesh, op2)
import sys

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


We now define the time-step size and the duration of the simulation. 

In [2]:
dt = 5.0

tmax = 9000.
dumpfreq = int(tmax / (9*dt))


Build a volume mesh. We again start by defining a periodic interval mesh in the horizontal and extrude this into the vertical dimension to create a three-dimensional mesh.

In [3]:
nlayers = 70  # horizontal layers
columns = 180  # number of columns
L = 144000.
m = PeriodicIntervalMesh(columns, L)

# build volume mesh
H = 35000.  # Height position of the model top
ext_mesh = ExtrudedMesh(m, layers=nlayers, layer_height=H/nlayers)
Vc = VectorFunctionSpace(ext_mesh, "DG", 2)
coord = SpatialCoordinate(ext_mesh)
x = Function(Vc).interpolate(as_vector([coord[0], coord[1]]))
a = 1000.
xc = L/2.
x, z = SpatialCoordinate(ext_mesh)
hm = 1.
zs = hm*a**2/((x-xc)**2 + a**2)

UFL:WARNING Discontinuous Lagrange element requested on interval * interval, creating DQ element.


We now invoke the smooth_z variable, I assume to smooth out the vertical profile somehow? We then apply this smoothing to the defined mesh.

In [5]:
dirname = 'non_hydro_mountain'

smooth_z = True
if smooth_z:
    dirname += '_smootherz'
    zh = 5000.
    xexpr = as_vector([x, conditional(z < zh, z + cos(0.5*pi*z/zh)**6*zs, z)])
else:
    xexpr = as_vector([x, z + ((H-z)/H)*zs])

new_coords = Function(Vc).interpolate(xexpr)
mesh = Mesh(new_coords)

Define the output. We will record the full velocity fields, as well as the perturbations to the potential temperature and density.

In [6]:
output = OutputParameters(dirname=dirname,
                          dumpfreq=dumpfreq,
                          dumplist=['u'],
                          perturbation_fields=['theta', 'rho'],
                          log_level='INFO')


Set up the corresponding state:

In [7]:
parameters = CompressibleParameters(g=9.80665, cp=1004.)
diagnostic_fields = [CourantNumber(), VelocityZ()]

state = State(mesh,
              dt=dt,
              output=output,
              parameters=parameters,
              diagnostic_fields=diagnostic_fields)


gusto:INFO Physical parameters that take non-default values:
gusto:INFO g: 9.80665, cp: 1004.0


Sponge layer stuff

In [8]:
# sponge function
sponge = SpongeLayerParameters(H=H, z_level=H-10000, mubar=0.15/dt)

Define the equations to solve, which are the compressible Euler equations. Set up the initial conditions for the velocity, density, and potential temperature. Also, define the function spaces we want for these variables.

In [9]:
eqns = CompressibleEulerEquations(state, "CG", 1, sponge=sponge)

# Initial conditions
u0 = state.fields("u")
rho0 = state.fields("rho")
theta0 = state.fields("theta")

# spaces
Vu = state.spaces("HDiv")
Vt = state.spaces("theta")
Vr = state.spaces("DG")

UFL:WARNING Discontinuous Lagrange element requested on interval * interval, creating DQ element.


Set up the relevant parameters for the test case.

In [10]:
# Thermodynamic constants required for setting initial conditions
# and reference profiles
g = parameters.g
N = parameters.N
p_0 = parameters.p_0
c_p = parameters.cp
R_d = parameters.R_d
kappa = parameters.kappa

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
x, z = SpatialCoordinate(mesh)
Tsurf = 300.
thetab = Tsurf*exp(N**2*z/g)
theta_b = Function(Vt).interpolate(thetab)

Determine the Exner pressure using the hydrostatic balance function.

In [11]:
# Calculate hydrostatic Pi
Pi = Function(Vr)
rho_b = Function(Vr)

piparams = {'ksp_type': 'gmres',
            'ksp_monitor_true_residual': None,
            'pc_type': 'python',
            'mat_type': 'matfree',
            'pc_python_type': 'gusto.VerticalHybridizationPC',
            # Vertical trace system is only coupled vertically in columns
            # block ILU is a direct solver!
            'vert_hybridization': {'ksp_type': 'preonly',
                                   'pc_type': 'bjacobi',
                                   'sub_pc_type': 'ilu'}}

compressible_hydrostatic_balance(state, theta_b, rho_b, Pi,
                                 top=True, pi_boundary=0.5,
                                 params=piparams)


    Residual norms for pisolver_ solve.
    0 KSP preconditioned resid norm 2.222800680832e+02 true resid norm 2.069609815553e+06 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 4.948737286376e-10 true resid norm 5.773621453311e-08 ||r(i)||/||b|| 2.789714954926e-14


More hydrostatic balance set up for initial conditions

In [12]:
def minimum(f):
    fmin = op2.Global(1, [1000], dtype=float)
    op2.par_loop(op2.Kernel("""
static void minify(double *a, double *b) {
    a[0] = a[0] > fabs(b[0]) ? fabs(b[0]) : a[0];
}
""", "minify"), f.dof_dset.set, fmin(op2.MIN), f.dat(op2.READ))
    return fmin.data[0]


p0 = minimum(Pi)
compressible_hydrostatic_balance(state, theta_b, rho_b, Pi,
                                 top=True, params=piparams)
p1 = minimum(Pi)
alpha = 2.*(p1-p0)
beta = p1-alpha
pi_top = (1.-beta)/alpha
compressible_hydrostatic_balance(state, theta_b, rho_b, Pi,
                                 top=True, pi_boundary=pi_top, solve_for_rho=True,
                                 params=piparams)


    Residual norms for pisolver_ solve.
    0 KSP preconditioned resid norm 3.315508603246e+02 true resid norm 4.100966222339e+06 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 7.272609795767e-10 true resid norm 6.886044476117e-08 ||r(i)||/||b|| 1.679127333117e-14
    Residual norms for pisolver_ solve.
    0 KSP preconditioned resid norm 3.315519283875e+02 true resid norm 4.100985947269e+06 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 7.262285720407e-10 true resid norm 5.373655419915e-08 ||r(i)||/||b|| 1.310332561245e-14
    Residual norms for rhosolver_ solve.
    0 KSP preconditioned resid norm 3.249820714969e-02 true resid norm 1.098372412418e+02 ||r(i)||/||b|| 1.000000000000e+00
    1 KSP preconditioned resid norm 1.470419708849e-14 true resid norm 1.012143512285e-12 ||r(i)||/||b|| 9.214939312404e-15
    Residual norms for rhosolver_ solve.
    0 KSP preconditioned resid norm 4.552413859466e-07 true resid norm 2.216763267285e-03 ||r(

Project the initial conditions

In [13]:
theta0.assign(theta_b)
rho0.assign(rho_b)
u0.project(as_vector([10.0, 0.0]))
remove_initial_w(u0)

state.set_reference_profiles([('rho', rho_b),
                              ('theta', theta_b)])

Set up the transport schemes

In [14]:
supg = True
if supg:
    theta_opts = SUPGOptions()
else:
    theta_opts = EmbeddedDGOptions()
transported_fields = [ImplicitMidpoint(state, "u"),
                      SSPRK3(state, "rho"),
                      SSPRK3(state, "theta", options=theta_opts)]

Set up the linear solver and time-stepper

In [15]:
# Set up linear solver
linear_solver = CompressibleSolver(state, eqns)

# build time stepper
stepper = CrankNicolson(state, eqns, transported_fields,
                        linear_solver=linear_solver)

Run!

In [ ]:
stepper.run(t=0, tmax=tmax)

/home/timandrews/firedrake/src/firedrake/firedrake/checkpointing.py:89: DeprecationWarning: DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.
  warnings.warn("DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.",
gusto:INFO at start of timestep, t=0, dt=5.0
gusto:INFO at start of timestep, t=5.0, dt=5.0
gusto:INFO at start of timestep, t=10.0, dt=5.0
gusto:INFO at start of timestep, t=15.0, dt=5.0
gusto:INFO at start of timestep, t=20.0, dt=5.0
gusto:INFO at start of timestep, t=25.0, dt=5.0
gusto:INFO at start of timestep, t=30.0, dt=5.0
gusto:INFO at start of timestep, t=35.0, dt=5.0
gusto:INFO at start of timestep, t=40.0, dt=5.0
gusto:INFO at start of timestep, t=45.0, dt=5.0
gusto:INFO at start of timestep, t=50.0, dt=5.0
gusto:INFO at start of timestep, t=55.0, dt=5.0
gusto:INFO at start of timestep, t=60.0, dt=5.0
gusto:INFO at start of timestep, t=65.0, dt=5.0
gusto:INFO at start of timestep, t=70